In [315]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')


In [316]:
df_merged = pd.read_csv("data\_2024\_merged\merged_data.csv")
df_merged 

,valid_time,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp
0,01-01-24,18.80,72.8,300.28345,298.79312,0.0,0.0,101279.910
1,01-01-24,19.05,72.8,300.91122,298.72922,0.0,0.0,101281.370
2,02-01-24,18.80,72.8,300.19160,298.49875,0.0,0.0,101224.320
3,02-01-24,19.05,72.8,300.74103,298.40060,0.0,0.0,101227.850
4,03-01-24,18.80,72.8,299.84067,298.13846,0.0,0.0,101160.195
...,...,...,...,...,...,...,...,...
727,29-12-24,19.05,72.8,300.16570,298.23690,0.0,0.0,101466.620
728,30-12-24,18.80,72.8,300.97717,298.98740,0.0,0.0,101442.690
729,30-12-24,19.05,72.8,301.34230,299.03720,0.0,0.0,101448.500
730,31-12-24,18.80,72.8,300.60540,299.00990,0.0,0.0,101487.400


In [317]:
df_merged

,valid_time,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp
0,01-01-24,18.80,72.8,300.28345,298.79312,0.0,0.0,101279.910
1,01-01-24,19.05,72.8,300.91122,298.72922,0.0,0.0,101281.370
2,02-01-24,18.80,72.8,300.19160,298.49875,0.0,0.0,101224.320
3,02-01-24,19.05,72.8,300.74103,298.40060,0.0,0.0,101227.850
4,03-01-24,18.80,72.8,299.84067,298.13846,0.0,0.0,101160.195
...,...,...,...,...,...,...,...,...
727,29-12-24,19.05,72.8,300.16570,298.23690,0.0,0.0,101466.620
728,30-12-24,18.80,72.8,300.97717,298.98740,0.0,0.0,101442.690
729,30-12-24,19.05,72.8,301.34230,299.03720,0.0,0.0,101448.500
730,31-12-24,18.80,72.8,300.60540,299.00990,0.0,0.0,101487.400


In [318]:
df_merged["diff_temperature"] = df_merged["max_temperature"] - df_merged["mean_temperature"]


In [319]:
df_merged

,valid_time,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp,diff_temperature
0,01-01-24,18.80,72.8,300.28345,298.79312,0.0,0.0,101279.910,1.49033
1,01-01-24,19.05,72.8,300.91122,298.72922,0.0,0.0,101281.370,2.18200
2,02-01-24,18.80,72.8,300.19160,298.49875,0.0,0.0,101224.320,1.69285
3,02-01-24,19.05,72.8,300.74103,298.40060,0.0,0.0,101227.850,2.34043
4,03-01-24,18.80,72.8,299.84067,298.13846,0.0,0.0,101160.195,1.70221
...,...,...,...,...,...,...,...,...,...
727,29-12-24,19.05,72.8,300.16570,298.23690,0.0,0.0,101466.620,1.92880
728,30-12-24,18.80,72.8,300.97717,298.98740,0.0,0.0,101442.690,1.98977
729,30-12-24,19.05,72.8,301.34230,299.03720,0.0,0.0,101448.500,2.30510
730,31-12-24,18.80,72.8,300.60540,299.00990,0.0,0.0,101487.400,1.59550


In [320]:
# Ensure valid_time is in datetime format
df_merged["valid_time"] = pd.to_datetime(df_merged["valid_time"], format="%d-%m-%y", errors="coerce")

# Extract numeric features from date
df_merged["month"] = df_merged["valid_time"].dt.month  # Extract month (1-13)
df_merged["day_of_year"] = df_merged["valid_time"].dt.dayofyear 
df_merged["year"] = df_merged["valid_time"].dt.year         # Extract Year
 # Extract day of year (1-365)

# Drop the original datetime column
df_merged.drop(columns=["valid_time"], inplace=True)

In [321]:
df_merged

,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp,diff_temperature,month,day_of_year,year
0,18.80,72.8,300.28345,298.79312,0.0,0.0,101279.910,1.49033,1,1,2024
1,19.05,72.8,300.91122,298.72922,0.0,0.0,101281.370,2.18200,1,1,2024
2,18.80,72.8,300.19160,298.49875,0.0,0.0,101224.320,1.69285,1,2,2024
3,19.05,72.8,300.74103,298.40060,0.0,0.0,101227.850,2.34043,1,2,2024
4,18.80,72.8,299.84067,298.13846,0.0,0.0,101160.195,1.70221,1,3,2024
...,...,...,...,...,...,...,...,...,...,...,...
727,19.05,72.8,300.16570,298.23690,0.0,0.0,101466.620,1.92880,12,364,2024
728,18.80,72.8,300.97717,298.98740,0.0,0.0,101442.690,1.98977,12,365,2024
729,19.05,72.8,301.34230,299.03720,0.0,0.0,101448.500,2.30510,12,365,2024
730,18.80,72.8,300.60540,299.00990,0.0,0.0,101487.400,1.59550,12,366,2024


In [322]:
conditions = [
    (df_merged['max_temperature'] >= 310.15),
    (df_merged['max_temperature'] < 310.15),
    (df_merged['diff_temperature'] >= 4.5),
    (df_merged['diff_temperature'] < 4.5),
]

values = [
    '1',
    '0',
    '1',
    '0',
]

df_merged['heat_label'] = np.select(conditions, values)

In [323]:
df_merged

,latitude,longitude,max_temperature,mean_temperature,max_precipitation,mean_precipitation,mslp,diff_temperature,month,day_of_year,year,heat_label
0,18.80,72.8,300.28345,298.79312,0.0,0.0,101279.910,1.49033,1,1,2024,0
1,19.05,72.8,300.91122,298.72922,0.0,0.0,101281.370,2.18200,1,1,2024,0
2,18.80,72.8,300.19160,298.49875,0.0,0.0,101224.320,1.69285,1,2,2024,0
3,19.05,72.8,300.74103,298.40060,0.0,0.0,101227.850,2.34043,1,2,2024,0
4,18.80,72.8,299.84067,298.13846,0.0,0.0,101160.195,1.70221,1,3,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...
727,19.05,72.8,300.16570,298.23690,0.0,0.0,101466.620,1.92880,12,364,2024,0
728,18.80,72.8,300.97717,298.98740,0.0,0.0,101442.690,1.98977,12,365,2024,0
729,19.05,72.8,301.34230,299.03720,0.0,0.0,101448.500,2.30510,12,365,2024,0
730,18.80,72.8,300.60540,299.00990,0.0,0.0,101487.400,1.59550,12,366,2024,0


In [324]:
if os.path.exists("data/_2010/_labelled/heat_labelled.csv"):
    os.remove("data/_2010/_labelled/heat_labelled.csv")
    # df_merged.to_csv("data/_2024/_labelled/heat_labelled.csv")
    # print("File created") 